In [1]:
!pip install dspy

     ---------------------------------------- 0.0/174.6 kB ? eta -:--:--
     ---------------- ---------------------- 71.7/174.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 174.6/174.6 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/197.5 kB ? eta -:--:--
   ------------------------ --------------- 122.9/197.5 kB ? eta -:--:--
   ---------------------------------------- 197.5/197.5 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/407.5 kB ? eta -:--:--
   ---------------------------- ----------- 286.7/407.5 kB 8.9 MB/s eta 0:00:01
   ---------------------------------------- 407.5/407.5 kB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.9 MB 9.6 MB/s eta 0:00:01
   ------------- -

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.4 requires pydantic<2.0, but you have pydantic 2.5.0 which is incompatible.


In [4]:
import sys
import os
import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [5]:
llama = dspy.HFClientTGI(model="meta-llama/Llama-2-13b-chat-hf", port=[7140, 7141, 7142, 7143], max_tokens=150)
colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# # NOTE: After you finish this notebook, you can use GPT-3.5 like this if you like.
# turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct')
# # In that case, make sure to configure lm=turbo below if you choose to do that.

dspy.settings.configure(rm=colbertv2, lm=llama)

In [ ]:
train = [('<Golden Context>', 'Generate a ChatGPT Prompt for <Question>', 'Optimized ChatGPT Prompt')]

train = [dspy.Example(context=context, question=question, prompt=prompt).with_inputs('context, question') for context, question, prompt in train]

dev = [('<Golden Context>', 'Generate a ChatGPT Prompt for <Question>', 'Optimized ChatGPT Prompt')]

dev = [dspy.Example(context=context, question=question, prompt=prompt).with_inputs('context, question') for context, question, prompt in dev]

In [ ]:
# Define a dspy.Predict module with the signature `question -> answer` (i.e., takes a question and outputs an answer).
predict = dspy.Predict('context, question -> prompt')

class CoT(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()

        # here we declare the chain of thought sub-module, so we can later compile it (e.g., teach it a prompt)
        self.generate_answer = dspy.ChainOfThought('context, question -> prompt')
    
    def forward(self, question):
        return self.generate_answer(question=question)  # here we use the module

In [ ]:
gpt4T = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=1000, model_type='chat')

# See how well the 
def metric(gold, pred):

    gold_answer, pred_prompt = gold.answer, pred.prompt

    engaging = "Does the assessed text make for a self-contained, engaging tweet?"
    correct = f"The text should answer `{pred_prompt}` with `{gold_answer}`. Does the assessed text contain this answer?"
    
    with dspy.context(lm=gpt4T):
        pred_answer = dspy.Predict('prompt -> answer')(prompt=pred_prompt)
        correct = dspy.Predict('question, answer -> assessment')(question=correct, answer=pred_answer)

        # can implement other metrics here

    correct = [m.assessment_answer.lower() == 'yes' for m in [correct, engaging]]
    score = (correct) if correct else 0

    return score 

In [ ]:

metric_EM = metric

teleprompter = BootstrapFewShot(metric=metric_EM, max_bootstrapped_demos=2)
cot_compiled = teleprompter.compile(CoT(), trainset=train)

In [ ]:
# we can test by asking
cot_compiled(context='new golden context', question='Generate a ChatGPT Prompt for <Question>')

In [ ]:
NUM_THREADS = 32
evaluate_optimizer = Evaluate(devset=dev, metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=15)

evaluate_optimizer(cot_compiled)